In [13]:
pip install telethon

In [14]:
import asyncio
from telethon import TelegramClient
import re
import pandas as pd

api_id = ''
api_hash = ''
channel_username = '@fantasy_jedi'

year_pattern = re.compile(
    r'(?:год рождения|г\.р\.?|родился в|р\.)\s*(\d{4})|\b(19\d{2}|20[0-2]\d)\b',
    re.IGNORECASE
)

async def parse_users():
    data = []
    async with TelegramClient('session_name', api_id, api_hash) as client:
        async for message in client.iter_messages(channel_username, limit=2000):
            if not message.sender:
                continue

                try:
                    user = await client.get_entity(message.sender.id)
                    full_user = await client.get_full_user(user.id)
                except Exception as e:
                    print('Unknown')
                    continue

                bio = getattr(full_user, 'about', '') if full_user else ''
                match = year_pattern.search(bio)
                year = (match.group(1) or match.group(2)) if match else None

                if year:
                    data.append({
                        "first_name": user.first_name or "",
                        "last_name": user.last_name or "",
                        "username": user.username or "",
                        "year": year
                    })

        df = pd.DataFrame(data)
        print(df)

await parse_users()

Empty DataFrame
Columns: []
Index: []


In [15]:
year_pattern = re.compile(r'(?:^|[^0-9])(19\d{2}|20[0-2]\d)(?:$|[^0-9])')

async def parse_users():
    data2 = []
    async with TelegramClient('session_name', api_id, api_hash) as client:
        async for message in client.iter_messages(channel_username, limit=2000):
            if not message.sender:
                continue

            user = message.sender
            username = user.username or ""

            match = year_pattern.search(username)
            if match:
                year = match.group(1)
                data2.append({
                    "username": f"@{username}",
                    "year": year,
                })

    df2 = pd.DataFrame(data2)
    print(df2)

await parse_users()

Empty DataFrame
Columns: []
Index: []
